# Tokenizing a mealy machine from hoa using simple_hoa option

In [ ]:
hoa = 'HOA: v1\nStates: 4\nStart: 0\nAP: 10 ""o0"" ""o1"" ""o2"" ""o3"" ""o4"" ""i0"" ""i1"" ""i2"" ""i3"" ""i4""\nacc-name: all\nAcceptance: 0 t\nproperties: trans-labels explicit-labels state-acc deterministic\ncontrollable-AP: 5 6 7 8 9\n--BODY--\nState: 0\n[0&!1&!5&!6&!7&!8&9] 1\n[!0&!5&!6&!7&!8&9] 2\n[0&1&!5&!6&!7&!8&9] 3\nState: 1\n[0&!5&!6&!7&!8&9] 0\n[!0&!5&!6&!7&!8&9] 3\nState: 2\n[!1&!5&!6&7&!8&9] 0\n[!0&1&!5&!6&7&!8&9] 2\n[0&1&!5&!6&7&!8&9] 3\nState: 3\n[0&!5&!6&!7&!8&9] 0\n[!0&!5&!6&!7&!8&9] 3\n--END--'

In [ ]:
from ml2.mealy.mealy_machine import MealyMachine, HoaHeader
from ml2.mealy.mealy_tokenizer import MealyToSeqTokenizer


m = MealyMachine.from_hoa(hoa)
tokenizer = MealyToSeqTokenizer(
    method="simplified_hoa",
    include_body_tokens=True,
    pad=128,
    inputs=m.header.inputs,
    outputs=m.header.outputs,
)


In [ ]:
tokens = tokenizer.encode_tokens(m)
"".join(tokens)

In [ ]:
m = tokenizer.decode_tokens(tokens)
p = m.to_hoa(realizable=False)
print(p)
print(hoa)

# Tokenizing a mealy machine from hoa using transitions

In [ ]:
hoa = 'HOA: v1\nStates: 2\nStart: 0\nAP: 10 "i0" "i1" "i2" "i3" "i4" "o0" "o1" "o2" "o3" "o4"\nacc-name: all\nAcceptance: 0 t\nproperties: trans-labels explicit-labels state-acc deterministic\ncontrollable-AP: 5 6 7 8 9\n--BODY--\nState: 0\n[!2&5&6&!7&!8&9 | 2&!5&!6&!7&!8&9] 1\nState: 1\n[!4&!5&!6&!7&!8&9 | 4&5&!6&!7&!8&9] 1\n--END--'

In [ ]:
from ml2.mealy.mealy_machine import MealyMachine, HoaHeader
from ml2.mealy.mealy_tokenizer import MealyToSeqTokenizer


m = MealyMachine.from_hoa(hoa)
tokenizer = MealyToSeqTokenizer(
    method="transitions",
    notation="prefix",
    pad=128,
    inputs=m.header.inputs,
    outputs=m.header.outputs,
)


In [ ]:
tokens = tokenizer.encode_tokens(m)
"".join(tokens)

In [ ]:
m = tokenizer.decode_tokens(tokens)
p = m.to_hoa(realizable=False)
print(p)
print(hoa)

In [ ]:
del tokenizer

#  Model Checking

We can use spot in a container to model check mealy machines

In [ ]:
from ml2.ltl.ltl_spec import DecompLTLSpec
from ml2.datasets.utils import from_csv_str, to_csv_str
from ml2.mealy import MealyMachine
from ml2.tools.spot import Spot
from ml2.ltl.ltl_syn import LTLSynStatus
from ml2.tools.ltl_tool.tool_ltl_mc_problem import ToolLTLMCProblem, ToolLTLMCSolution

fields = {
    "assumptions": "(X (G ((! (o0)) | (((! (i4)) & (! (i3))) U ((! (i4)) & (i3)))))),(G (F (i4)))",
    "guarantees": "(G ((i0) -> (o4))),(G (((! (i1)) & (! (i0))) -> (F (((! (o0)) & (! (o2))) & (! (o1)))))),(G (F ((o2) <-> (X (o2))))),(G ((i1) -> (F (o4)))),(G ((! (o1)) | (! (o0)))),(G (((i0) & (X (i4))) -> (F ((o0) & (o1))))),(G ((o3) -> (X ((i1) R (((i1) -> (o0)) & ((! (i1)) -> (o1))))))),(G ((i2) -> (F (o0)))),(G (((o0) & (X ((! (i2)) & (! (o0))))) -> (X ((i2) R (! (o0)))))),(G ((i2) -> (o2)))",
    "realizable": 1,
    "hoa": 'HOA: v1\nStates: 2\nStart: 0\nAP: 10 ""i0"" ""i1"" ""i2"" ""i3"" ""i4"" ""o0"" ""o1"" ""o2"" ""o3"" ""o4""\nacc-name: all\nAcceptance: 0 t\nproperties: trans-labels explicit-labels state-acc deterministic\ncontrollable-AP: 5 6 7 8 9\n--BODY--\nState: 0\n[!2&5&6&!7&!8&9 | 2&!5&!6&!7&!8&9] 1\nState: 1\n[!4&!5&!6&!7&!8&9 | 4&5&!6&!7&!8&9] 1\n--END--',
    "inputs": "i0,i1,i2,i3,i4",
    "outputs": "o0,o1,o2,o3,o4",
}


formula = DecompLTLSpec.from_csv_fields(fields)

mealy = MealyMachine.from_hoa(from_csv_str(fields["hoa"]))

status =LTLSynStatus.from_csv_fields(fields=fields)


verifier = Spot()

verifier.model_check(ToolLTLMCProblem(parameters={}, realizable=True, specification=formula, mealy_machine=mealy)).status


In [ ]:
del verifier